In [ ]:
import logging
import math
import os
import sys
from pathlib import Path

import tomli
import numpy as np
import structlog

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style('whitegrid')

import pandas as pd
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)

In [ ]:
# Set the logging level
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [ ]:
import pytanis
from pytanis import GSheetsClient, PretalxClient
from pytanis.pretalx import subs_as_df, reviews_as_df, speakers_as_df

In [ ]:
# Be aware that this notebook might only run with the following version
pytanis.__version__

In [ ]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)

In [ ]:
pd.DataFrame(subs)

In [ ]:
pretalx_client = PretalxClient(blocking=True)
subs_count, subs = pretalx_client.submissions(cfg['event_name'], params={'questions': 'all'})
spkrs_count, spkrs = pretalx_client.speakers(cfg['event_name'], params={'questions': 'all'})
revs_count, revs = pretalx_client.reviews(cfg['event_name'])
subs, revs, spkrs = list(subs), list(revs), list(spkrs)

In [ ]:
subs_df = subs_as_df(subs, with_questions=True)
revs_df = reviews_as_df(revs)
spkrs_df = speakers_as_df(spkrs, with_questions=True)

# Speaker Statistics

Determine the number of speakers per company.

In [ ]:
# Group spkrs_df by "Q: Company / Institute" and count the number of submission and return a dataframe
company_counts = spkrs_df.groupby('Q: Company / Institute').size().reset_index(name='count')

# Sort the dataframe by the count of submissions
company_counts = company_counts.sort_values('count', ascending=False)

# rename "Q: Company / Institute" to "company_name"
company_counts = company_counts.rename(columns={'Q: Company / Institute': 'company_name'})

# company_counts.to_csv('company_counts.csv', index=False)
company_counts.head(10)

In [ ]:
from difflib import get_close_matches

# Step 0: Create a dictionary to map similar names to a consistent name
company_names = company_counts['company_name'].unique()
company_name_map = {}
for company_name in company_names:
    company_names_exclude_name = [name for name in company_names if name != company_name]
    
    # except handling company name
    lookup_name = company_name.strip()

    # do lookup without company form
    if lookup_name.endswith('GmbH'):
        lookup_name = lookup_name[:-4]
    matches = get_close_matches(lookup_name, company_names_exclude_name, n=1, cutoff=.8)  # Adjust the cutoff as needed
    if matches:
        company_name_map[company_name] = matches[0]

    # do lookup without company form
    if not lookup_name.endswith('GmbH'):
        lookup_name = f'{lookup_name} GmbH'
    matches = get_close_matches(lookup_name, company_names_exclude_name, n=1, cutoff=.8)  # Adjust the cutoff as needed
    if matches:
        company_name_map[company_name] = matches[0]

    # do lookup with original company name
    matches = get_close_matches(company_name, company_names_exclude_name, n=1, cutoff=.8)  # Adjust the cutoff as needed
    if matches:
        company_name_map[company_name] = matches[0]

company_name_map

In [ ]:
from collections import defaultdict

# Step 1: Build an adjacency list to represent the graph
adjacency_list = defaultdict(set)
for key, value in company_name_map.items():
    adjacency_list[key].add(value)
    adjacency_list[value].add(key)

# Step 2: Perform a DFS or BFS to find all connected components
def find_groups(adjacency_list):
    visited = set()
    groups = []

    def dfs(node, group):
        visited.add(node)
        group.append(node)
        for neighbor in adjacency_list[node]:
            if neighbor not in visited:
                dfs(neighbor, group)

    for node in adjacency_list:
        if node not in visited:
            group = []
            dfs(node, group)
            groups.append(group)
    
    return groups

# Step 3: Get the grouped names
company_name_groups = find_groups(adjacency_list)

# Step 4: for each key in company_name_map find its group in company_name_groups
company_name_group_map = {}
for key, value in company_name_map.items():
    for group in company_name_groups:
        if key in group:
            company_name_group_map[key] = group
            break

# Step 5: Assign the group names if available
company_counts['company_name_grouping_arr'] = company_counts['company_name'].apply(lambda x: company_name_group_map.get(x, [x]))

company_counts['company_name_grouping_str'] = company_counts['company_name_grouping_arr'].apply(lambda x: '[' + ', '.join(x) + ']')

company_counts.head(10)

In [ ]:
# Group "company_counts" by company_name_grouping column and sum the count column
company_counts_by_group = company_counts.groupby('company_name_grouping_str')['count'].sum().reset_index()

# Sort the dataframe by the count of submissions
company_counts_by_group = company_counts_by_group.sort_values('count', ascending=False)

# company_counts_by_group.to_csv('company_counts_by_group.csv', index=False)
company_counts_by_group

# Submission Statistics
Determine number of speakers/submissions per track and main track.

In [ ]:
# filter for all submitted talks
talks_df = subs_df.loc[subs_df['State'] == 'submitted']

# TODO: fetch main tracks from pretalx
main_tracks = ['PyData', 'PyCon', 'General']

# TODO: fetch tracks from pretalx
all_tracks = ['PyCon: MLOps & DevOps', 'PyCon: Programming & Software Engineering', 'PyCon: Python Language & Ecosystem', 'PyCon: Security', 'PyCon: Testing', 'PyCon: Django & Web', 'PyData: Data Handling & Engineering', 'PyData: Machine Learning & Deep Learning & Statistics', 'PyData: Natural Language Processing & Audio (incl. Generative AI NLP)', 'PyData: Computer Vision (incl. Generative AI CV)', 'PyData: Generative AI', 'PyData: Embedded Systems & Robotics', 'PyData: PyData & Scientific Libraries Stack', 'PyData: Visualisation & Jupyter', 'PyData: Research Software Engineering', 'General: Community & Diversity', 'General: Education, Career & Life', 'General: Ethics & Privacy', 'General: Infrastructure - Hardware & Cloud', 'General: Rust', 'General: Others']

# all available submission types
submission_types = talks_df['Submission type'].unique()

# all available expertise levels
expertise_levels = list(talks_df['Q: Expected audience expertise: Domain'].unique()) + list(talks_df['Q: Expected audience expertise: Python'].unique())
expertise_levels = list(set(expertise_levels))

# all expertise categories
expertise_categories = ['Q: Expected audience expertise: Python', 'Q: Expected audience expertise: Domain']

# create an dataframe with 'all_tracks' and all 'submission_types' as rows
tracks_df = pd.DataFrame(all_tracks, columns=['Track'])

### All independent of submission type

In [ ]:
# group submittaded talks by track and count the number of submissions
talks_quantification_by_domain_expertise = talks_df.groupby(['Track', 'Q: Expected audience expertise: Domain']).size().unstack(fill_value=0)
talks_quantification_by_domain_expertise = tracks_df.join(talks_quantification_by_domain_expertise, on='Track')
talks_quantification_by_domain_expertise['Total'] = talks_quantification_by_domain_expertise[['None', 'Novice', 'Intermediate', 'Advanced']].sum(axis=1)
talks_quantification_by_domain_expertise['Total %'] = (talks_quantification_by_domain_expertise['Total'] / talks_quantification_by_domain_expertise['Total'].sum() * 100).round(2)
talks_quantification_by_domain_expertise['Main Track'] = talks_quantification_by_domain_expertise['Track'].apply(lambda x: x.split(':')[0] if ':' in x else x)
talks_quantification_by_domain_expertise['Total % per Main Track'] = talks_quantification_by_domain_expertise.groupby('Main Track')['Total'].transform(lambda x: (x / x.sum() * 100).round(2))

# reorder columns
talks_quantification_by_domain_expertise = talks_quantification_by_domain_expertise[['Main Track', 'Track', 'Total', 'Total %', 'Total % per Main Track', 'None', 'Novice', 'Intermediate', 'Advanced']]
talks_quantification_by_python_expertise = talks_df.groupby(['Track', 'Q: Expected audience expertise: Python']).size().unstack(fill_value=0)
talks_quantification_by_python_expertise = tracks_df.join(talks_quantification_by_python_expertise, on='Track')
talks_quantification_by_python_expertise['Main Track'] = talks_quantification_by_python_expertise['Track'].apply(lambda x: x.split(':')[0] if ':' in x else x)
talks_quantification_by_python_expertise = talks_quantification_by_python_expertise[['Main Track', 'Track', 'None', 'Novice', 'Intermediate', 'Advanced']]

# join talks_quantification_by_domain_expertise and talks_quantification_by_python_expertise and keep add a group column name fir the expertise level
talks_quantification = pd.merge(talks_quantification_by_domain_expertise, talks_quantification_by_python_expertise, on=['Main Track', 'Track'], how='outer')

talks_quantification.columns = pd.MultiIndex.from_tuples([
    ('', col) if (col == 'Track') | (col == 'Total') | (col == 'Total %') | (col == 'Total % per Main Track') | (col == 'Main Track') else 
    ('Expected Domain Expertise by Audience', col.rstrip("_xy")) if col.endswith('_x') else 
    ('Expected Python Expertise by Audience', col.rstrip("_xy")) 
    for col in talks_quantification.columns
])

# fill NaN values with 0
talks_quantification.fillna(0, inplace=True)

talks_quantification

In [ ]:
# Compress overall table for plotting
talks_quantification_condensed = talks_quantification.copy()

talks_quantification_condensed['', 'Expected Domain Expertise by Audience'] = talks_quantification_condensed['Expected Domain Expertise by Audience'].to_numpy().tolist()
talks_quantification_condensed['', 'Expected Python Expertise by Audience'] = talks_quantification_condensed['Expected Python Expertise by Audience'].to_numpy().tolist()

talks_quantification_condensed = talks_quantification_condensed.drop(columns=['Expected Domain Expertise by Audience', 'Expected Python Expertise by Audience'], level=0)
talks_quantification_condensed.columns = talks_quantification_condensed.columns.droplevel(0)

# helper functions for plotting
def cell_histogram_with_labels(values, global_max_value=None):
    max_value = max(values) if global_max_value is None else global_max_value  # Maximalwert für Skalierung
    bar_heights = [100 / len(values)] * len(values)  # Gleichmäßige Balkenhöhen (in Prozent)
    bars = ""
    labels = ['None', 'Novice', 'Intermediate', 'Advanced']
    for i, value in enumerate(values):
        label = labels[i]
        bar_width = (value / max_value) * 100 if max_value > 0 else 0  # Width
        y_position = i * bar_heights[0]  # Y-Position of each bar
        # Rechteck (Bar)
        bars += f'<rect x="0" y="{y_position}%" width="{bar_width}%" height="{bar_heights[0]}%" style="fill:#d65f5f50;" />'
        # Text (Label)
        bars += f'<text x="2" y="{y_position + bar_heights[0] / 1.8}%" dominant-baseline="middle" font-size="10" fill="black">{label} ({int(value)})</text>'
    
    svg = f"""
    <svg width="100" height="50" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 50">
        {bars}
    </svg>
    """
    return svg

def single_value_histogram(value, max_value):
    # Calculate the width of the bar as a percentage
    bar_width = (value / max_value) * 100 if max_value > 0 else 0
    
    # Generate the SVG
    svg = f"""
    <svg width="100" height="50" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 20">
        <!-- Rectangle (Bar) -->
        <rect x="0" y="-20" width="{bar_width}%" height="80" style="fill:#d65f5f50;" />
        <!-- Text (Label) -->
        <text x="5" y="15" font-size="14" fill="black">{round(value, 2)}%</text>
    </svg>
    """
    return svg

# Generate output
title = f'All {int(talks_quantification_condensed['Total'].sum())} submitted talks, long talks and tutorials (excluding pending and withdrawn submissions) <br> ****'

talks_quantification_condensed_styled = talks_quantification_condensed.style \
    .set_caption(title) \
    .set_table_styles([
        {'selector': 'caption', 'props': [('font-family', 'Arial'), ('font-size', '20px'), ('font-weight', 'bold')]},
        {'selector': 'th', 'props': [('font-family', 'Arial'), ('max-width', '160px')]}
    ]) \
    .set_properties(**{'font-family': 'Arial'}) \
    .format({
        ('Total'): '{:.0f}',
        ('Total %'): lambda value: single_value_histogram(
            value,
            talks_quantification_condensed['Total %'].max()
        ),
        ('Total % per Main Track'): lambda value: single_value_histogram(
            value,
            talks_quantification_condensed['Total % per Main Track'].max()
        ),
        'Expected Domain Expertise by Audience': lambda values: cell_histogram_with_labels(
            values,
            np.concatenate(talks_quantification_condensed['Expected Domain Expertise by Audience'].to_numpy()).max()
        ),
        'Expected Python Expertise by Audience': lambda values: cell_histogram_with_labels(
            values,
            np.concatenate(talks_quantification_condensed['Expected Python Expertise by Audience'].to_numpy()).max()
        ),
    })

talks_quantification_condensed_styled.to_html('talks_quantification.html', index=False, escape=False)

talks_quantification_condensed_styled

# Future Todos
- Compare against historical events
- Split by submission type
- Make independent of submission type
- Topic modelling and keyword freqzency on submissions
- Fetch tracks from Pretalx automatically (no static list as above)